In [ ]:
import numpy as np
with open('input.txt') as file:
    paths = file.read().split('\n')
starting_point = np.array([500, 0])

In [ ]:
import time
from IPython.display import display, clear_output

def print_cave_slice(cave_slice, clear=False):
    string = ''
    for row in np.transpose(cave_slice):
        for char in row:
            string += char
        string += '\n'
    if clear:
        clear_output(wait=True)
    print(string)
    time.sleep(0.05)

In [ ]:
# returns True if position is free and false otherwise
# returns None if position is outside of cave_slice
def is_free(position, cave_slice):
    if not (all(position >= 0) and all(position < cave_slice.shape)):
        return None
    return cave_slice[tuple(position)] == '.'

# returns the new position or None if movement was not possible inside cave_slice
def move_sand(position, cave_slice):
    moves = np.array([[0,1], [-1,0], [2,0]])
    new_position = position.copy()
    for move in moves:
        new_position += move
        return_value = is_free(new_position, cave_slice)
        if type(return_value) == type(None):
            return None
        if return_value:
            return new_position
    return position

In [ ]:
def draw_cave(all_paths, x_min, x_max, y_min, y_max):
    cave_slice = np.empty((x_max-x_min+1, y_max+1), dtype=str)
    cave_slice.fill('.')
    for path in all_paths:
        for i_line in range(len(path)-1):
            x_coords = [path[i_line,0] - x_min, path[i_line+1,0] - x_min]
            y_coords = [path[i_line,1], path[i_line+1,1]]
            x0 = min(x_coords)
            x1 = max(x_coords)
            y0 = min(y_coords)
            y1 = max(y_coords)
            cave_slice[x0:x1+1,y0:y1+1] = '#'
    return cave_slice

In [ ]:
def fill_cave(cave_slice, starting_point, printing=False):
    n_sand_particles = 0
    done = False
    while not done:
        start_position = starting_point - np.array([x_min, 0])
        cave_slice[tuple(start_position)] = '+'
        old_position = np.array(start_position)
        n_sand_particles += 1
        sand_is_placed = False
        while not sand_is_placed:
            new_position = move_sand(old_position, cave_slice)
            if type(new_position) == type(None):
                n_sand_particles -= 1
                done = True
                break
            if all(new_position == start_position):
                done = True
                break
            if all(new_position == old_position):
                sand_is_placed = True
                break
            cave_slice[tuple(old_position)] = '.'
            cave_slice[tuple(new_position)] = '+'
            old_position = new_position

            if printing:
                print_cave_slice(cave_slice, clear=True)
    if printing:
        print_cave_slice(cave_slice, clear=True)
    return n_sand_particles

# Import

In [ ]:
all_paths = [0] * len(paths)
for i_path, path in enumerate(paths):
    splitted = path.split(' -> ')
    this_path = [0] * len(splitted)
    for i, straight_line in enumerate(splitted):
        this_path[i] = list(eval(straight_line))
    all_paths[i_path] = np.array(this_path)

In [ ]:
# min max from all paths
all_chords = np.concatenate(all_paths)
x_min = all_chords[:,0].min()
x_max = all_chords[:,0].max()
y_min = all_chords[:,1].min()
y_max = all_chords[:,1].max()

# Part One

In [ ]:
cave_slice = draw_cave(all_paths, x_min, x_max, y_min, y_max)
n_sand = fill_cave(cave_slice, starting_point, printing=False)
print(n_sand)

# Part Two

In [ ]:
# add bottom line
y_max += 2
left_right_margin = y_max + 1
x_min = min(starting_point[0] - left_right_margin, x_min)
x_max = max(starting_point[0] + left_right_margin, x_max)
all_paths.append(np.array([[x_min, y_max],[x_max, y_max]]))

In [ ]:
cave_slice = draw_cave(all_paths, x_min, x_max, y_min, y_max)
n_sand = fill_cave(cave_slice, starting_point, printing=False)
print(n_sand)

In [ ]:
print_cave_slice(cave_slice)